In [ ]:
# https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf
# !pip install --upgrade --user ase
# !pip install ipywidgets 
# !pip install ipympl
# !conda install nglview -c conda-forge -y
# !pip install --upgrade --user asap3
# !pip install nglview
# !pip install scikit-learn
# !pip install scipy
# !pip install sgdml
# !pip install sgdml[ase]
# !pip install sgdml[torch]
# !conda install -c conde-forge kimpy openkim-models
# !jupyter-nbextension enable nglview --py --sys-prefix
%load_ext autoreload
%autoreload 2
from utils import *
%matplotlib widget

# Gaussian Process 

## Univariate normal distribution


$$
\mathcal{N}(\mu, \sigma^2)
$$

$$
p(x \mid \mu, \sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp{ \left( -\frac{(x - \mu)^2}{2\sigma^2}\right)}
$$

In [ ]:
def univariate_normal(x, mean, variance):
    """pdf of the univariate normal distribution."""
    return ((1. / np.sqrt(2 * np.pi * variance)) * 
            np.exp(-(x - mean)**2 / (2 * variance)))

In [ ]:
# set up plot
univariate_plot(univariate_normal)

## Multivariate normal distribution

$$
p(\mathbf{x} \mid \mathbf{\mu}, \Sigma) = \frac{1}{\sqrt{(2\pi)^d \lvert\Sigma\rvert}} \exp{ \left( -\frac{1}{2}(\mathbf{x} - \mathbf{\mu})^T \Sigma^{-1} (\mathbf{x} - \mathbf{\mu}) \right)}
$$

$$
\mathcal{N}(\mathbf{\mu}, \Sigma)
$$

In [ ]:
def multivariate_normal(x, d, mean, covariance):
    """pdf of the multivariate normal distribution."""
    x_m = x - mean
    return (1. / (np.sqrt((2 * np.pi)**d * np.linalg.det(covariance))) * 
            np.exp(-(np.linalg.solve(covariance, x_m).T.dot(x_m)) / 2))

## 2D Multivariate normal distribution
$$
\mathcal{N}\left(
\begin{bmatrix}
0 \\
0
\end{bmatrix}, 
\begin{bmatrix}
1 & 0 \\
0 & 1 
\end{bmatrix}\right)
$$

$$
\mathcal{N}\left(
\begin{bmatrix}
0 \\
1
\end{bmatrix}, 
\begin{bmatrix}
1 & 0.8 \\
0.8 & 1
\end{bmatrix}\right)
$$

In [ ]:
plt.close('all')

multivariate_plot(multivariate_normal, nb_of_x=100)

## Marginal and Conditional normal distributions

If both $\mathbf{x}$ and $\mathbf{y}$ are [jointly normal](https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Joint_normality) random vectors defined as:
$$
\begin{bmatrix}
\mathbf{x} \\
\mathbf{y} 
\end{bmatrix}
\sim
\mathcal{N}\left(
\begin{bmatrix}
\mu_{\mathbf{x}} \\
\mu_{\mathbf{y}}
\end{bmatrix},
\begin{bmatrix}
A & C \\
C^T & B
\end{bmatrix}
\right)
= \mathcal{N}(\mu, \Sigma)
, \qquad 
\Sigma^{-1} = \Lambda = 
\begin{bmatrix}
\tilde{A} & \tilde{C} \\
\tilde{C}^T & \tilde{B}
\end{bmatrix}
$$

The [conditional distribution](https://en.wikipedia.org/wiki/Conditional_probability_distribution) of $\mathbf{x}$ given $\mathbf{y}$ is defined as:

$$
p(\mathbf{x} \mid \mathbf{y}) = \mathcal{N}(\mu_{x|y}, \Sigma_{x|y})
$$

With:
$$\begin{split}
\Sigma_{x|y} & = A - CB^{-1}C^\top = \tilde{A}^{-1} \\
\mu_{x|y} & = \mu_x + CB^{-1}(\mathbf{y}-\mu_y)
\end{split}$$



### Proof

$
\begin{split}
p(x_1, x_2) = exp
\left[
-\frac{1}{2}
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}^T
\begin{pmatrix}
\Sigma_{11} & \Sigma_{12} \\ 
\Sigma_{21} & \Sigma_{22}
\end{pmatrix}^{-1}
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}
\right]
\end{split}
$

By using thr following idendty:

$
\begin{split}
M^{-1}=
\begin{pmatrix}
A & B \\ 
C & D
\end{pmatrix}^{-1}=
\begin{pmatrix}
I & 0 \\ 
-C^{-1}C & I
\end{pmatrix}
\begin{pmatrix}
(M/D)^{-1} & 0 \\ 
0 & D^{-1}
\end{pmatrix}
\begin{pmatrix}
I & -BD^{-1} \\ 
0 & I
\end{pmatrix}
\end{split}
$

We obtainig the folowing:

$
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}^T
\begin{pmatrix}
\Sigma_{11} & \Sigma_{12} \\ 
\Sigma_{21} & \Sigma_{22}
\end{pmatrix}^{-1}
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}=
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}^T
\begin{pmatrix}
I & 0 \\ 
-\Sigma_{22}^{-1} \Sigma_{21} & I
\end{pmatrix}
\begin{pmatrix}
(\Sigma/\Sigma_{22})^{-1} & 0 \\ 
0 & \Sigma_{22}^{-1}
\end{pmatrix}
\begin{pmatrix}
I & -\Sigma_{12}\Sigma_{22}^{-1} \\ 
0 & I
\end{pmatrix}
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}
$

And finally:

$
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2)) + (x_2-\mu_2)^T\Sigma_22^{-1}(x_2-\mu_2)
$

And plugin back into the first Eq:

$
\begin{split}
p(x_1, x_2) = exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2)) + (x_2-\mu_2)^T\Sigma_22^{-1}(x_2-\mu_2)
\right]
\end{split}
$
$
\begin{split}
p(x_1, x_2) = exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))
\right] \cdot
exp
\left[
-\frac{1}{2}
(x_2-\mu_2)^T\Sigma_22^{-1}(x_2-\mu_2)
\right]
\end{split}
$

and from the axiom of probability we obtain the follwoing equlaity:

$
\begin{split}
p(x_1, x_2) = p(x_1|x_2)p(x_2)= exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))
\right] \cdot
exp
\left[
-\frac{1}{2}
(x_2-\mu_2)^T\Sigma_22^{-1}(x_2-\mu_2)
\right] = 
exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))
\right] \cdot
p(x_2)
\end{split} \\ 
\Rightarrow p(x_1|x_2) = exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))
\right] \\
\mu_{1|2} = \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2)
\Sigma_{1|2} = (\Sigma/\Sigma_{22})=\Sigma_{11}-\Sigma_{12}\Sigma_{22}^{-1}\Sigma_{21}
$

In [ ]:
plt.close('all')
condition_plot(nb_of_x=100)

## Gaussian Process
Gaussian process (GP) Is a method predicting $y^*$ for a given $x^*$ and getting: $y_i=f(x_i )$. 
GP assumes that p(f(x_1),...,f(x_N  )) is jointly Gaussian, i.e., the value of a new point m is defined as a multidimensional gaussian with $\mu(x)$ and $\Sigma(x)$, where $\Sigma(x)$ is calculated as $\Sigma_{ij}=k(x_i,x_j)$, and $k$ is the kernel function.<br/>
Which can be regarded as a "distance" function which defines how strongly the value f(x_i) is coupled to point f(x_j). 
When trying to predict a new point x^* we use prior data points, calculating the new kernel values, and finally obtaining the following new multidimensional Gaussian distribution:

$$
\begin{pmatrix}
f \\ 
f^*
\end{pmatrix}~
\mathcal{N}
\begin{pmatrix}
\begin{pmatrix}
\mu \\ 
\mu^*
\end{pmatrix},
\begin{pmatrix}
k & k^*\\ 
{k^{*}}^{T} & k^{**}
\end{pmatrix}
\end{pmatrix}
$$
<br/>
$f$ - Vector of all observed $y_i$ values, $f=y_i=f(x_i)$.<br/>
$f^*$ - Prediction function for point $x^*$, $f^*=y^*=f(x^*)$.<br/>
$\mu$ - Vector of all observed mean values.<br/>
$\mu^*$- Mean values for the prediction for x^*. <br/>
$k$ - Covariance matrix of all observed points<br/>
$k^*$ - Kernel vector $k^*=k(x^*,x_i )$<br/>
$k^{**}$ - Self kernel vector $k^{**}=k(x^*,x^* )$<br/>


### Applying the Conditional 
$$f(x^*)=\mu^* +{{k}^{*}}^{T}  k^{-1}(y-\mu)$$ <br/>
For simplicity, we can define the mean values to be zero<br/>
$$f(x^*) ={k^*}^{T}k^{-1}y=\sum_{i=1}^{N}\alpha_ik(x_i, x^*)$$

### The Most Popular Kernal
Here, we will use the squared exponential kernel, also known as Gaussian kernel or RBF kernel
$$
\kappa(\mathbf{x}_i,\mathbf{x}_j) = \sigma_f^2 \exp\left(-\frac{1}{2l^2}
  (\mathbf{x}_i - \mathbf{x}_j)^T
  (\mathbf{x}_i - \mathbf{x}_j)\right)\tag{10}
$$

In [ ]:
# https://juanitorduz.github.io/gaussian_process_reg/
# http://krasserm.github.io/2018/03/19/gaussian-processes/
def kernel(X1, X2, l=1.0, sigma_f=1.0):
    '''
    Isotropic squared exponential kernel. Computes
    a covariance matrix from points in X1 and X2.

    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        Covariance matrix (m x n).
    '''
    sqdist = np.sum(X1 ** 2, 1).reshape(-1, 1) + np.sum(X2 ** 2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f ** 2 * np.exp(-0.5 / l ** 2 * sqdist)

### The Conditional on A Nosey Dataset
If we have a training dataset with noisy function values $\mathbf{y} = \mathbf{f} + \boldsymbol\epsilon$ where noise $\boldsymbol\epsilon \sim \mathcal{N}(\mathbf{0}, \sigma_y^2 \mathbf{I})$ is independently added to each observation then the predictive distribution is given by

$$
\begin{align*}
p(\mathbf{f}_* \lvert \mathbf{X}_*,\mathbf{X},\mathbf{y}) &= \mathcal{N}(\mathbf{f}_* \lvert \boldsymbol{\mu}_*, \boldsymbol{\Sigma}_*) \\
\boldsymbol{\mu_*} &= \mathbf{K}_*^T \mathbf{K}_y^{-1} \mathbf{y} \\
\boldsymbol{\Sigma_*} &= \mathbf{K}_{**} - \mathbf{K}_*^T \mathbf{K}_y^{-1} \mathbf{K}_*
\end{align*}
$$
where $\mathbf{K}_y = \mathbf{K} + \sigma_y^2\mathbf{I}$.

In [ ]:
def posterior_predictive(X_s, X_train, Y_train, l=1.0, sigma_f=1.0, sigma_y=1e-8):
    '''
    Computes the suffifient statistics of the GP posterior predictive distribution
    from m training data X_train and Y_train and n new inputs X_s.

    Args:
        X_s: New input locations (n x d).
        X_train: Training locations (m x d).
        Y_train: Training targets (m x 1).
        l: Kernel length parameter.
        sigma_f: Kernel vertical variation parameter.
        sigma_y: Noise parameter.

    Returns:
        Posterior mean vector (n x d) and covariance matrix (n x n).
    '''
    K = kernel(X_train, X_train, l, sigma_f) + sigma_y ** 2 * np.eye(len(X_train))
    K_s = kernel(X_train, X_s, l, sigma_f)
    K_ss = kernel(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
    K_inv = inv(K)


    alphas = K_inv.dot(Y_train)
    mu_s = K_s.T.dot(alphas)


    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)

    return mu_s, cov_s

In [ ]:
plt.close()
noise = 0.1
x = [-3, -2 , -1,  1, 2 , 3, 4]
def f(x):
    func = np.exp(np.sin(x))
    return func
    
gaussian_process(x, f, noise, posterior_predictive, kernel)

In [ ]:
plt.close('all')
noise = 0.1
kernal = ConstantKernel(1.0) * RBF(length_scale=1.0)

gaussian_process_interactive(f, noise, kernal)

## GP Force Field 
a straightforward formulation of a vector-valued estimator
takes the form:
$$
\mathbf{\hat{f}}=\begin{bmatrix}\hat{f}_1(x), \dots, \hat{f}_N\end{bmatrix}^{T}
$$
$\mathbf{\hat{f}}: \mathbb{R}^N\rightarrow\mathbb{R}^N$ where each component: ${\hat{f}_i}: \mathbb{R}^N\rightarrow\mathbb{R}$

In [ ]:
plt.close('all')
from ase_utils import *
"""Demonstrates molecular dynamics with constant temperature."""


a = 3.519
atoms_type = 'Ni'
atoms = Atoms([Atom(atoms_type, (0, 0, 0)), 
               Atom(atoms_type, (0, a/2, a/2)),
               Atom(atoms_type, (a/2, 0, a/2)),
               Atom(atoms_type, (a/2, a/2, 0))])
atoms.set_cell([a, a, a])
atoms.set_pbc(True)
atoms = atoms*2
atoms_copy = atoms.copy()

ngl.view_ngl(atoms, w=500, h=500)

In [ ]:
from ase.calculators.emt import EMT
from ase.calculators.kim.kim import KIM
T = 300  # Kelvin
# Describe the interatomic interactions with the Effective Medium Theory
# atoms.calc = EMT()
atoms.calc = KIM("EAM_Dynamo_MishinFarkasMehl_1999_Ni__MO_400591584784_005")
# Set the momenta corresponding T
MaxwellBoltzmannDistribution(atoms, T * 4 * units.kB)

In [ ]:
# A molecular dynamics object will operate on the atoms by moving them according to their forces - 
# it integrates Newton’s second law numerically
# # Room temperature simulation
dyn = Langevin(atoms, 1 * units.fs, 2*T * units.kB, 0.002)
dyn.attach(printenergy(atoms), interval=500)
# We also want to save the positions of all atoms after every 100th time step.
traj = Trajectory('reference.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)

In [ ]:
# Now run the dynamics
steps = 30000 - 1
dyn.run(steps)

In [ ]:
traj = Trajectory('reference.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

Instead of mapping to scalar outputs, we can alternatively model the covariance
function as a matrix $k : \chi × \chi → \mathbb{R}^N\rightarrow\mathbb{R}^{N\times N}$ that expresses the interaction among
multiple output components. Together with a vector-valued mean function $\mu : \chi →
\mathbb{R}^N$ , we can then sample realizations of vector-valued functions from the GP
$$
\mathbf{\hat{f}} ∼ \mathcal{GP}\begin{bmatrix}\mathbf{\mu(x)}, \mathbf{k(x,x^{'})}\end{bmatrix}
$$

### the navie approch:
$$
\mathbf{\hat{f}}=\begin{bmatrix}\hat{f}_1(x), \dots, \hat{f}_N\end{bmatrix}^{T}
$$
$$
\hat{f}_i(x) = \mathcal{GP}\begin{bmatrix}\mathbf{\mu(x)_i}, \mathbf{k(x,x^{'})_i}\end{bmatrix}
$$
$$
\mathbf{\hat{f}}=\begin{bmatrix}\hat{f}_1(x) \\ \dots \\ \hat{f}_N(x)\end{bmatrix}
=\begin{bmatrix}\mathcal{GP}\begin{bmatrix}\mathbf{\mu(x)_1}, \mathbf{k(x,x^{'})_1}\end{bmatrix} \\ 
\dots
\\ \mathcal{GP}\begin{bmatrix}\mathbf{\mu(x)_N}, \mathbf{k(x,x^{'})_N}\end{bmatrix}\end{bmatrix}
$$

In [ ]:
traj = Trajectory('reference.traj', 'r', atoms)
start, every = 1, 20
(x, y), (x_test, y_test) = md_dataset_split(traj, start, every)
print(f'train points: {len(x)}')
noise = 0.2

In [ ]:
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
kernal= ConstantKernel(1.0) * RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=kernal, alpha=noise**2)
gpr.fit(x, y)
plot_gpr(gpr, x, y, x_test, y_test)

In [ ]:
from ase.calculators.calculator import Calculator
class GPRCalculator(Calculator):
    implemented_properties = ['forces']

    def __init__(self, gdr, *args, **kwargs):
        super(GPRCalculator, self).__init__(*args, **kwargs)
        self.gpr_model = gdr

    def calculate(self, atoms=None, *args, **kwargs):
        super(GPRCalculator, self).calculate(atoms, *args, **kwargs)
        r = np.array(atoms.get_positions())
        x = r.reshape([1, -1])
        f = self.gpr_model.predict(x)
        self.results = {'forces': f.reshape(-1, 3)}

In [ ]:
atoms = atoms_copy.copy()
atoms.calc = GPRCalculator(gpr)
MaxwellBoltzmannDistribution(atoms, T * 2 * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002)
steps = 30000
traj = Trajectory('moldyn_gdr.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)
dyn.run(steps)
traj = Trajectory('moldyn_gdr.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

#### some tricks
##### change the kernal:
##### The Matérn kernel
For our application, we considered a subclass from the parametric
Matérn family (22–24) of (isotropic) kernel functions
$$
k: C_{\mu=n+\frac{1}{2}}(d)=exp{-\frac{\sqrt{2\nu}d}{\sigma}}P_{n}(d)
$$
$$
P_n(d)=\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}}\begin{pmatrix}n\\k\end{pmatrix}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}
$$

In [ ]:
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern
kernal= Matern(length_scale=1.0, nu=2.5)
gpr = GaussianProcessRegressor(kernel=kernal, alpha=noise**2)
gpr.fit(x, y)
plot_gpr(gpr, x, y, x_test, y_test)

In [ ]:
atoms = atoms_copy.copy()
atoms.calc = GPRCalculator(gpr)
MaxwellBoltzmannDistribution(atoms, T * 2 * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002)
steps = 30000
traj = Trajectory('moldyn_gdr.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)
dyn.run(steps)
traj = Trajectory('moldyn_gdr.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

#### normalize y
$$
\hat{y}_{train} = \frac{y_{train}-E[y_{train}]}{\sqrt{Var[y_{train}]}}
$$

In [ ]:
kernal= Matern(length_scale=1.0, nu=2.5)
gpr = GaussianProcessRegressor(kernel=kernal, alpha=noise**2, normalize_y=True)
gpr.fit(x, y)
plot_gpr(gpr, x, y, x_test, y_test)

In [ ]:
atoms = atoms_copy.copy()
atoms.calc = GPRCalculator(gpr)
MaxwellBoltzmannDistribution(atoms, T * 2 * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002)
steps = 30000
traj = Trajectory('moldyn_gdr.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)
dyn.run(steps)
traj = Trajectory('moldyn_gdr.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

### Roto-translational invariance
Covariance functions remain valid under any transformation of their domain
) is again a kernel function. A rather trivial implication is that all
invariances of that input transformation are inherited, providing yet another opportunity
to characterize the properties of the predictor [92].

The so-called Coulomb matrix representation [7] goes one step further and represents
each pair of nuclei in terms their Coulomb interaction instead of a simple distance. The
Coulomb energy is the only nuclei-nuclei interaction term in the Hamiltonian and empirically a good starting point for inference about molecular properties [9]. We use a slight
variation of this descriptor for our purpose, whereby atoms of different type interact on a
normalized scale,
$$
D_{ij}=\begin{matrix}\lVert{r_i - r_j}\lVert & i > j \\ 0 & i \leq j \end{matrix} 
$$

In [ ]:
d, d_test = x_to_d(x), x_to_d(x_test)
kernal= Matern(length_scale=1.0, nu=2.5)
gpr = GaussianProcessRegressor(kernel=kernal, alpha=noise**2, normalize_y=True)
gpr.fit(d, y)
plot_gpr(gpr, d, y, d_test, y_test)

In [ ]:
class iGPRCalculator(Calculator):
    implemented_properties = ['forces']

    def __init__(self, gdr, *args, **kwargs):
        super(iGPRCalculator, self).__init__(*args, **kwargs)
        self.gpr_model = gdr

    def calculate(self, atoms=None, *args, **kwargs):
        super(iGPRCalculator, self).calculate(atoms, *args, **kwargs)
        r = np.array(atoms.get_positions())
        x = r.reshape([1, -1])
        f = self.gpr_model.predict(x_to_d(x))
        self.results = {'forces': f.reshape(-1, 3)}

In [ ]:
atoms = atoms_copy.copy()
atoms.calc = iGPRCalculator(gpr)
MaxwellBoltzmannDistribution(atoms, T * 2 * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002)
steps = 30000
traj = Trajectory('moldyn_gdr.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)
dyn.run(steps)
traj = Trajectory('moldyn_gdr.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

## incoding physical insight to the model
In this setting, the corresponding RKHS is vector-valued and it has been shown
that the representer theorem continues to hold. Each component of the kernel
function $k_{ij}$ specifies a covariance between a pair of outputs $f_i(x)$ and $f_j(x)$, which
makes it straightforward to impose linear constraints $g(x) = \hat{G}[\mathbf{f(x)}]$ on the GP
prior
$$
\mathbf{\hat{g}(x)} ∼ \mathcal{GP}\begin{bmatrix}\mathbf{\hat{G}\mu(x)}, \hat{G}\mathbf{k(x,x^{'})}{\hat{G}'}^{T}\end{bmatrix}
\\
for A, B\; linear\; operators\\
Cov[Ax,By]=ACov[x,y]B^{T}\\
E[Ax] = AE[x]
$$

Here, we aim to construct a GP that inherits the correct structure of a conservative
force field to ensure integrability, so that the corresponding energy potential can
be recovered from the same model. We start by considering, that the force field
estimator $\mathbf{\hat{f}_{F}(x)}$ and the PES estimator $\hat{f}_{E}(x)$ are related via some operator $\hat{G}$ . To
impose energy conservation, we require that the curl vanishes  for every
input to the transformed energy model:
$$
\nabla\times \hat{G}\begin{bmatrix}\hat{f}_E\end{bmatrix} = \mathbf{0}
$$
As expected, this is satisfied by the derivative operator Gˆ = ∇ or, in the case of
energies and forces, the negative gradient operator
$$
\mathbf{\hat{f}_{F}(x)}=\hat{G}\begin{bmatrix}\hat{f}_E\end{bmatrix}=-\nabla\hat{f}_E
$$

Since differentiation is a linear operator, the result is another GP with realizations $\mathbf{f_F}: \chi^{3N}\rightarrow \mathbb{R}^{3N}$
$$
\mathbf{\hat{f}_F(x)} ∼ \mathcal{GP}\begin{bmatrix}\mathbf{-\nabla\mu(x)}, \nabla_x\mathbf{k(x,x^{'})}{ \nabla_{x'}}^{T}\end{bmatrix}
$$

$$
\nabla\mathbf{k}{ \nabla}^{T}=Hess_x(k)
$$
$$
\begin{bmatrix}Hess_s(k)\end{bmatrix}_{ij}=\frac{\partial^2 k}{\partial x_i \partial x_j}
$$

This Hessian kernel gives rise to the following gradient domain machine
learning [88, 89] force model as the posterior mean of the corresponding GP:<br>
he trained force field estimator collects the contributions of the
partial derivatives 3N of all training points M to compile the prediction.
It takes the form
$$
\mathbf{\hat{f}_F(x)}=\sum_i^M{\sum_j^{3N}{(\mathbf{\alpha_i})_j\frac{\partial}{\partial x_j}\nabla k(\mathbf{x}, \mathbf{x_i})}}
$$
    
Because the trained model is a (fixed) linear combination of kernel functions,
integration only affects the kernel function itself. The corresponding expression for
the energy predictor
$$
\mathbf{\hat{f}_E(x)}=\sum_i^M{\sum_j^{3N}{(\mathbf{\alpha_i})_j\frac{\partial}{\partial x_j} k(\mathbf{x}, \mathbf{x_i})}}
$$

#### Roto-translational invariance
Covariance functions remain valid under any transformation of their domain $D : X → D,
i.e. k(D(x),D(x′)) = k_D(x,x′)$ is again a kernel function.
$$
D_{ij}=\begin{matrix}\lVert{r_i - r_j}\lVert & i > j \\ 0 & i \leq j \end{matrix} 
$$
and under the chain rule:
$$
\mathbf{k_F}=\nabla_x k_D \nabla_x^\top = \mathbf{J_D}^\top(\nabla_D k_D \nabla_D^\top)\mathbf{J_D}
$$
where:
$$
\nabla_x = \begin{pmatrix}\frac{\partial}{\partial x_1} \\ \dots \\ \frac{\partial}{\partial x_{3N}}\end{pmatrix}=
\begin{pmatrix}\frac{\partial D_{11}}{\partial x_1}\frac{\partial}{\partial  D_{11}} +
                \frac{\partial D_{12}}{\partial x_1}\frac{\partial}{\partial  D_{12}} + 
                \dots
                \frac{\partial D_{1N}}{\partial x_1}\frac{\partial}{\partial  D_{1N}} +
                \frac{\partial D_{23}}{\partial x_1}\frac{\partial}{\partial  D_{23}} +
                \dots
                \frac{\partial D_{NN}}{\partial x_1}\frac{\partial}{\partial  D_{NN}}\\
                \dots \\ 
                \frac{\partial D_{11}}{\partial x_{3N}}\frac{\partial}{\partial  D_{11}} +
                \frac{\partial D_{12}}{\partial x_{3N}}\frac{\partial}{\partial  D_{12}} + 
                \dots
                \frac{\partial D_{1N}}{\partial x_{3N}}\frac{\partial}{\partial  D_{1N}} +
                \frac{\partial D_{23}}{\partial x_{3N}}\frac{\partial}{\partial  D_{23}} +
                \dots
                \frac{\partial D_{NN}}{\partial x_{3N}}\frac{\partial}{\partial  D_{NN}}\end{pmatrix}=
\begin{pmatrix}\frac{\partial D_{11}}{\partial x_1} &
               \frac{\partial D_{12}}{\partial x_1} &  
               \dots &
               \frac{\partial D_{1N}}{\partial x_1} &
               \frac{\partial D_{23}}{\partial x_1} & 
               \dots &
               \frac{\partial D_{NN}}{\partial x_1} \\
               \dots & \dots & \dots & \dots & \dots & \dots \\ 
               \frac{\partial D_{11}}{\partial x_{3N}} & 
               \frac{\partial D_{12}}{\partial x_{3N}} &  
               \dots &
               \frac{\partial D_{1N}}{\partial x_{3N}} &  
               \frac{\partial D_{23}}{\partial x_{3N}} & 
               \dots &
               \frac{\partial D_{NN}}{\partial x_{3N}} \end{pmatrix}
               \begin{pmatrix}\frac{\partial}{\partial D_{11}} \\ \dots \\ \frac{\partial}{\partial D_{NN}}\end{pmatrix}
               =\mathbf{J_D}^\top\nabla_D \\
               \\
\mathbf{J_D}=\begin{matrix}\frac{\mathbf{r_i} - \mathbf{r_j} }{\lVert{\mathbf{r_i}  - \mathbf{r_j}}\lVert^3} & i > j \\ 0 & i \leq j \end{matrix} 
$$

In [ ]:
from sgdml.cli import _print_dataset_properties, _print_model_properties, _print_task_properties

dataset_path = 'reference'
dataset = from_traj(f'{dataset_path}.traj', overwrite=True)
dataset_path = f'{dataset_path}.npz'
# test = np.load('ethanol_dft.npz')
_print_dataset_properties(dataset)

In [ ]:
n_train = 250
n_valid = 1500
file = all_script(dataset_path, n_train, n_valid, sigs=range(1, 100, 1), use_cg=False, use_torch=True)
print(f'please run \"bash {file}\" from the terminal')

In [ ]:
model_path = 'reference-unknown-train200-sym1.npz'
traj = Trajectory('reference.traj', 'r', atoms)
start, every = 10, 10
plot_gdml(model_path, traj, start, every)

In [ ]:
from sgdml.intf.ase_calc import SGDMLCalculator
from ase_utils import *
from ase.optimize import QuasiNewton
import gc
model_path = 'reference-unknown-train100-sym1.npz'
calc = SGDMLCalculator(model_path)
atoms = atoms_copy.copy()
atoms.set_calculator(calc)
# set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(atoms, T * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002, trajectory='moldyn_sgdml.traj')    
dyn.attach(printenergy(atoms), interval=500)

# now run the dynamics
printenergy(atoms)
dyn.run(30000)
gc.collect()


In [ ]:
traj = Trajectory('moldyn_sgdml.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

### For scalar output
$$
\begin{bmatrix}f^*\end{bmatrix}^{1\times1} = \begin{bmatrix}{k^*}^T\end{bmatrix}^{1\times M}\begin{bmatrix}\begin{bmatrix}K^{-1}\end{bmatrix}^{M \times M}\begin{bmatrix}y\end{bmatrix}^{M\times 1}\end{bmatrix}^{M\times1}=\sum_{i}^{M}\alpha_i k(x_i,x^*)
$$

### For vector output
$$
\begin{bmatrix}\mathbf{f^*}\end{bmatrix}^{3N\times1} ={k^*}^{T}K^{-1}\mathbf{f}=\sum_i^M{\sum_j^{3N}{(\mathbf{\alpha_i})_j\frac{\partial}{\partial x_j}\nabla k(\mathbf{x}, \mathbf{x_i})}}
$$

$$
K=\nabla_{x} k(x,x') \nabla_{x^{'}}^\top = 
\begin{pmatrix}
\begin{pmatrix}\nabla_{x^{(1)}} k(x^{(1)},x^{(1)}) \nabla_{x^{(1)}}^\top\end{pmatrix} & \dots & \begin{pmatrix}\nabla_{x^{(M)}} k(x^{(M)},x^{(1)}) \nabla_{x^{(1)}}^\top\end{pmatrix} \\
\dots & \dots & \dots \\
\begin{pmatrix}\nabla_{x^{(M)}} k(x^{(M)},x^{(1)}) \nabla_{x^{(1)}}^\top\end{pmatrix} & \dots & \begin{pmatrix}\nabla_{x^{(M)}} k(x^{(M)},x^{(M)}) \nabla_{x^{(M)}}^\top\end{pmatrix}
\end{pmatrix}
\\
\nabla k(x^{(i)}, x^{(j)})\nabla ^\top=\begin{bmatrix}\frac{\partial}{\partial x^{(j)}_1 }\nabla k(x^{(i)},x^{(j)}),\dots,\frac{\partial}{\partial x^{(j)}_{3N} }\nabla k(x^{(i)},x^{(j)})\end{bmatrix}^{3N\times 3N}
=\begin{bmatrix}\frac{\partial}{\partial x^{(j)}_1 }\frac{\partial}{\partial x^{(i)}_{1}} k(x^{(i)},x^{(j)}) &\dots & \frac{\partial}{\partial x^{(j)}_{3N} }\frac{\partial}{\partial x^{(i)}_{1}} k(x^{(i)},x^{(j)}) \\
\dots & \dots & \dots \\
\frac{\partial}{\partial x^{(j)}_1}\frac{\partial}{\partial x^{(i)}_{3N}} k(x^{(i)},x^{(j)}) &\dots & \frac{\partial}{\partial x^{(j)}_{3N} }\frac{\partial}{\partial x^{(i)}_{3N}} k(x^{(i)},x^{(j)})
\end{bmatrix}^{3N\times 3N}
\\
\begin{bmatrix}K\end{bmatrix}^{M\cdot3N\times M\cdot3N} \;
\begin{bmatrix}\mathbf{f}\end{bmatrix}^{M\cdot3N\times 1}
\\
$$
$$
{k^*}^{T}
=\nabla_x k(x^{*}, x)\nabla_{x^{'}} ^\top 
=
\begin{bmatrix}
\begin{pmatrix}\nabla_{x^*} k(x^{*}, x^{(1)})\nabla_{x^{(1)}} ^\top\end{pmatrix} & 
\dots & 
\begin{pmatrix}\nabla k(x^{*}, x^{(M)})\nabla_{x^{(M)}} ^\top\end{pmatrix}
\end{bmatrix}^{3N\times M\cdot 3N}
= 
\begin{bmatrix}
\begin{pmatrix}\frac{\partial}{\partial x^{(1)}_1 }\nabla k(x^{*},x^{(1)}),\dots,\frac{\partial}{\partial x^{(1)}_{3N} }\nabla k(x^{*},x^{(1)})\end{pmatrix} & 
\dots & 
\begin{pmatrix}\frac{\partial}{\partial x^{(M)}_1 }\nabla k(x^{*},x^{(M)}),\dots,\frac{\partial}{\partial x^{(M)}_{3N} }\nabla k(x^{*},x^{(M)})\end{pmatrix}
\end{bmatrix}^{3N\times M\cdot 3N}
\\
\\
\begin{bmatrix}\mathbf{f^*}\end{bmatrix}^{3N\times1} =\begin{bmatrix}\begin{bmatrix}{k^*}^{T}\end{bmatrix}^{3N \times M \cdot 3N}\begin{bmatrix}\begin{bmatrix}K^{-1}\end{bmatrix}^{M\cdot 3N \times M\cdot 3N}\begin{bmatrix}\mathbf{f}\end{bmatrix}^{M\cdot 3N \times 1}\end{bmatrix}^{M \cdot 3N \times 1}\end{bmatrix}^{3N\times 1}
\\
\begin{bmatrix}\mathbf{f^*}\end{bmatrix}^{3N\times1} =
{k^*}^{T}
K^{-1}
\mathbf{f}=
\begin{pmatrix}
\begin{pmatrix}\frac{\partial}{\partial x^{(1)}_1 }\nabla k(x^{*},x^{(1)}),\dots,\frac{\partial}{\partial x^{(1)}_{3N} }\nabla k(x^{*},x^{(1)})\end{pmatrix} & 
\dots & 
\begin{pmatrix}\frac{\partial}{\partial x^{(M)}_1 }\nabla k(x^{*},x^{(M)}),\dots,\frac{\partial}{\partial x^{(M)}_{3N} }\nabla k(x^{*},x^{(M)})\end{pmatrix}
\end{pmatrix}
\underbrace{ K^{-1}\mathbf{f}}_{\alpha _{ij}}
$$

### The Matérn kernel
For our application, we considered a subclass from the parametric
Matérn family (22–24) of (isotropic) kernel functions
$$
k: C_{\nu=n+\frac{1}{2}}(d)=exp{-\frac{\sqrt{2\nu}d}{\sigma}}P_{n}(d)
$$
$$
P_n(d)=\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}}\begin{pmatrix}n\\k\end{pmatrix}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}
$$

### The full kernel

$$
\mathbf{k_F(x, x^{'})}=\nabla k(x, x^{'})\nabla ^\top=\begin{bmatrix}\frac{\partial}{\partial x^{'}_1 }\nabla k(x,x^{'}),\dots,\frac{\partial}{\partial x^{'}_{3N} }\nabla k(x,x^{'})\end{bmatrix}
$$
$$
=\begin{pmatrix}5(\mathbf{x-x^{'}})(\mathbf{x-x^{'}})^\top-\mathbb{1}\sigma(\sigma +\sqrt{5}d))\end{pmatrix}\frac{5}{3\sigma^{4}}exp\begin{pmatrix}-\frac{\sqrt{5}d}{\sigma}\end{pmatrix}
$$
$$
\mathbf{k_F(x, x^{'})}\in \mathbb{R}^{3N \times 3N}
$$

$$
\mathbf{k_E(x, x^{'})}=k(x, x^{'})\nabla ^\top
=5(\mathbf{x-x^{'}})(\sigma+d)\frac{5}{3\sigma^{3}}exp\begin{pmatrix}-\frac{\sqrt{5}d}{\sigma}\end{pmatrix}
$$
$$
\mathbf{k_E(x, x^{'})}\in \mathbb{R}^{1 \times 3N}
$$


### Matérn covariance derivatives
$$
k: C_{\nu=n+\frac{1}{2}}(d)=B(d)P_{n}(d)
\\
B(d) = exp{\begin{pmatrix}-\frac{\sqrt{2\nu}d}{\sigma}\end{pmatrix}}
\\
P_n(d)=\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}}\begin{pmatrix}n\\k\end{pmatrix}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}
$$

$$
\frac{\partial \kappa}{\partial x_i} = \frac{\partial P_n}{\partial x_i}B + \frac{\partial B}{\partial x_i}P_n
$$

$$
\frac{\partial P_n}{\partial x_i}=\sum_{k=0}^n{\frac{(n+k)!}{(2n)!}\begin{pmatrix}n\\k\end{pmatrix}\frac{(n-k)(x_i -x^{'}_i)}{d^2}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}}\\
$$

$$
\frac{\partial B}{\partial x_i} =-\frac{\sqrt{2\nu}(x_i -x^{'}_i)}{\sigma d}exp{-\frac{\sqrt{2\nu}d}{\sigma}}
$$

$$
\frac{\partial^2 \kappa}{\partial x_i \partial x_j} = 
B\frac{\partial^2 P_n}{\partial x_i \partial x_j} +
\frac{\partial B}{\partial x_i}\frac{\partial P_n}{\partial x_j} +
\frac{\partial B}{\partial x_j}\frac{\partial P_n}{\partial x_i} +
P_n\frac{\partial^2 B}{\partial x_i \partial x_j} 
$$

$$
\begin{bmatrix}\frac{\partial^2 P_n}{\partial x_i \partial x_j}\end{bmatrix}_{i\ne j}=
\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}\begin{pmatrix}n \\ k \end{pmatrix}
\frac{(n-k-2)(n-k)(x_i -x^{'}_i)(x_j - x^{'}_j)}{d^4}
\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}}
$$

$$
\begin{bmatrix}\frac{\partial^2 P_n}{\partial x_i \partial x_j}\end{bmatrix}_{i=j}=
\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}\begin{pmatrix}n \\ k \end{pmatrix}
\frac{(n-k-2)(n-k)(x_i -x^{'}_i)^2}{d^4}
\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}} + \sum_{k=0}^n{\frac{(n+k)!}{(2n)!}\begin{pmatrix}n\\k\end{pmatrix}\frac{(n-k)}{d^2}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}}\\
$$

$$
\begin{bmatrix}\frac{\partial^2 B}{\partial x_i \partial x_j}\end{bmatrix}_{i\ne j} =
\frac{\sqrt{2\nu}(x_i-x^{'})(x_j-x^{'}_j)(\sqrt{2\nu} d+\sigma)}{\sigma^2d^3}\exp{\begin{pmatrix}-\frac{\sqrt{2\nu}d}{\sigma}\end{pmatrix}}
$$

$$
\begin{bmatrix}\frac{\partial^2 B}{\partial x_i \partial x_j}\end{bmatrix}_{i = j} =
\frac{\sqrt{2\nu}(x_i-x^{'})(x_j-x^{'}_j)(\sqrt{2\nu} d+\sigma)}{\sigma^2d^3}\exp{\begin{pmatrix}-\frac{\sqrt{2\nu}d}{\sigma}\end{pmatrix}} - \frac{\sqrt{2\nu}(x_i -x^{'}_i)}{\sigma d}exp{\begin{pmatrix}-\frac{\sqrt{2\nu}d}{\sigma}\end{pmatrix}}
$$